In [1]:
import sys
import json
from opentrons import protocol_api
from opentrons.types import Point
import helpers as h
import opentrons.execute
import time

/var/user-packages/usr/lib/python3.7/site-packages/urllib3/__init__.py:38: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.7.4'. See: https://github.com/urllib3/urllib3/issues/3020
  exceptions.NotOpenSSLWarning,


Helper initialized in 0.00 seconds.


In [2]:
commands = h.pullexcel("format_test.xlsx")
print(commands)


Reading spreadsheet data...
Finished reading spreadsheet.
[('A1', 6000), ('A2', 5750), ('A3', 5500), ('A4', 5250), ('A5', 5000), ('A6', 4750), ('B1', 4500), ('B2', 4250), ('B3', 4000), ('B4', 3750), ('B5', 3500), ('B6', 3250), ('C1', 3000), ('C2', 2750), ('C3', 2500), ('C4', 2250), ('C5', 2000), ('C6', 1750), ('D1', 1500), ('D2', 1250), ('D3', 1000), ('D4', 750), ('D5', 500), ('D6', 250)]


In [ ]:
print("Setup has initiated.")
protocol = opentrons.execute.get_protocol_api('2.12')
l = h.Loader(protocol)
l.wipe_deck()
print("Homing...")
start = time.time()
protocol.home()
end = time.time()
print(f"Homing done in {end - start:.2f} seconds.")

#worker = h.Worker("format_test.xlsx", vol_col=1, last_letter="D", last_number=6)
commands = h.pullexcel("format_test.xlsx")
print(commands)


In [ ]:
# Labware setup
tips_1000 = l.load('opentrons_96_filtertiprack_1000ul', 11)

tips_300 = l.load("opentrons_96_tiprack_300ul", 10)

reservoir = l.load("rexchem_1_reservoir_225112.5ul.json", 4)

tubes = l.load("bohden_24_long.json", 5)


p300 = protocol.load_instrument("p300_single", "left", tip_racks=[tips_300])
p300.well_bottom_clearance.aspirate, p300.well_bottom_clearance.dispense = 2, 2
p300.default_speed = 200

p1000 = protocol.load_instrument("p1000_single_gen2", "right", tip_racks=[tips_1000])
p1000.well_bottom_clearance.aspirate, p1000.well_bottom_clearance.dispense = 2, 2
p1000.default_speed = 200

#if p20.has_tip:
#    print("Tip detected, dropping...")
#    p20.drop_tip()

print("Setup is done.")
#active.transfer(vol, reservoir["A1"], plate[job["location"]], new_tip="never", blow_out=True)


In [ ]:
# Command time
p300.drop_tip()
p1000.drop_tip()

p300.pick_up_tip()
p1000.pick_up_tip()

for entry in commands:
    loc, vol = entry
    active = p300
    if vol > 300:
        active = p1000
    print(f"Transferring {vol}uL to {loc} using {active.name}.")
    active.transfer(vol, reservoir["A1"], tubes[loc], new_tip="never")

p300.drop_tip(tips_300["A1"])
p1000.drop_tip(tips_1000["A1"])
protocol.home()